In [1]:
import pandas as pd
import numpy as np
import time
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [2]:
# Switch over to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Read in datasets

# trainData = pd.read_csv('interactions_train.csv')
# validationData = pd.read_csv('interactions_validation.csv')
# testData = pd.read_csv('interactions_test.csv')

In [4]:
# Convert dataset to tensors
class InteractionDataset(Dataset):
    def __init__(self, df):
        self.users = torch.tensor(df['u'].values, dtype=torch.long)
        self.recipes = torch.tensor(df['i'].values, dtype=torch.long)
        self.ratings = torch.tensor(df['rating'].values, dtype=torch.float)

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        return self.users[idx], self.recipes[idx], self.ratings[idx]

In [5]:
# Convert dataset to InteractionDataset
trainInteractionDataset = InteractionDataset(trainData)
valInteractionDataset = InteractionDataset(valData)
testInteractionDataset = InteractionDataset(testData)

In [6]:
# Setup DataLoaders for training, validating and testing
def setupLoaders(batchSize, trainInteractionDataset, valInteractionDataset, testInteractionDataset):
    trainLoader = DataLoader(trainInteractionDataset, batch_size=batchSize, shuffle=True)
    valLoader = DataLoader(valInteractionDataset, batch_size=batchSize, shuffle=False)
    testLoader = DataLoader(testInteractionDataset, batch_size=batchSize, shuffle=False)

    return trainLoader, valLoader, testLoader

In [7]:
# Neural Network
class BasicNN(nn.Module):
    def __init__(self, n_users, n_recipes, n_factors=10, hidden_size=20):
        super(BasicNN, self).__init__()

        # Embedding layers
        self.user_factors = nn.Embedding(n_users, n_factors)
        self.recipe_factors = nn.Embedding(n_recipes, n_factors)

        # Dense layers
        self.fc1 = nn.Linear(n_factors * 2, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

        # Initializing embeddings
        nn.init.xavier_uniform_(self.user_factors.weight)
        nn.init.xavier_uniform_(self.recipe_factors.weight)

    def forward(self, user, recipe):
        user_embedding = self.user_factors(user)
        recipe_embedding = self.recipe_factors(recipe)

        # Concatenate the embeddings
        x = torch.cat([user_embedding, recipe_embedding], dim=1)

        # Pass through dense layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x.squeeze()

class DeepNN(nn.Module):
    def __init__(self, n_users, n_recipes, n_factors=20, hidden_sizes=[100, 80, 60, 40, 20], dropout_rate=0.5):
        super(DeepNN, self).__init__()

        # Embedding layers
        self.user_factors = nn.Embedding(n_users, n_factors)
        self.recipe_factors = nn.Embedding(n_recipes, n_factors)

        # Initializing embeddings
        nn.init.xavier_uniform_(self.user_factors.weight)
        nn.init.xavier_uniform_(self.recipe_factors.weight)

        # Dense layers
        layer_sizes = [n_factors * 2] + hidden_sizes + [1]
        self.layers = nn.ModuleList()
        for i in range(len(layer_sizes) - 1):
            self.layers.append(nn.Linear(layer_sizes[i], layer_sizes[i + 1]))
            if i < len(layer_sizes) - 2:  # no activation, dropout, or normalization after the last layer
                self.layers.append(nn.ReLU())
                self.layers.append(nn.Dropout(dropout_rate))
                self.layers.append(nn.BatchNorm1d(layer_sizes[i + 1]))

    def forward(self, user, recipe):
        user_embedding = self.user_factors(user)
        recipe_embedding = self.recipe_factors(recipe)

        # Concatenate the embeddings
        x = torch.cat([user_embedding, recipe_embedding], dim=1)

        # Pass through dense layers
        for layer in self.layers:
            x = layer(x)

        return x.squeeze()

In [8]:
# Max number of Users and Recipe IDs
maxUsersId = max(trainData['u'].max(), valData['u'].max(), testData['u'].max())
maxRecipesId = max(trainData['i'].max(), valData['i'].max(), testData['i'].max())
maxUsersId, maxRecipesId

(25075, 178264)

In [9]:
def trainEpoch(model, trainLoader, device, criterion, optimizer, displayProgression):
    model.train()
    trainLoss = 0.0
    threshold = 4.0
    TP_Train = 0
    FP_Train = 0
    TN_Train = 0
    FN_Train = 0
    for batch, (users, recipes, ratings) in enumerate(trainLoader):
        users = users.to(device)
        recipes = recipes.to(device)
        ratings = ratings.to(device)

        optimizer.zero_grad(set_to_none=True)
        predictions = model(users, recipes)
        loss = criterion(predictions, ratings)
        loss.backward()
        optimizer.step()
        trainLoss += loss.item()

        predictedPositives = (predictions >= threshold).float()
        actualPositives = (ratings >= threshold).float()
        actualNegatives = (ratings < threshold).float()

        TP_Train += (predictedPositives * actualPositives).sum().item()
        FP_Train += (predictedPositives * actualNegatives).sum().item()
        TN_Train += ((1 - predictedPositives) * actualNegatives).sum().item()
        FN_Train += ((1 - predictedPositives) * actualPositives).sum().item()

        if displayProgression and batch % (len(trainLoader) // 10) == 0 and batch < 10 * (len(trainLoader) // 10):
            trainAccuracy = (TP_Train + TN_Train) / (TP_Train + TN_Train + FP_Train + FN_Train)
            trainPrecision = (TP_Train / (TP_Train + FP_Train)) if (TP_Train + FP_Train) != 0 else 0
            current = (batch + 1) * len(ratings)
            print(f"Training Loss: {loss.item():>.7f} [{current:>6d}/{(len(trainLoader.dataset)):>6d}] Accuracy: {(100*trainAccuracy):>3.1f} Precision: {(100*trainPrecision):>3.1f}")

    trainAccuracy = (TP_Train + TN_Train) / (TP_Train + TN_Train + FP_Train + FN_Train)
    trainPrecision = (TP_Train / (TP_Train + FP_Train)) if (TP_Train + FP_Train) != 0 else 0
    trainLoss /= len(trainLoader)
    return trainLoss, trainAccuracy, trainPrecision

In [10]:
def testEpoch(model, trainLoader, device, criterion, displayProgression):
    model.eval()
    valLoss = 0.0
    threshold = 4.0
    TP_Val = 0
    FP_Val = 0
    TN_Val = 0
    FN_Val = 0
    if displayProgression:
        print()
    with torch.no_grad():
        for batch, (users, recipes, ratings) in enumerate(valLoader):
            users = users.to(device)
            recipes = recipes.to(device)
            ratings = ratings.to(device)

            predictions = model(users, recipes)
            loss = criterion(predictions, ratings)
            valLoss += loss.item()

            predictedPositives = (predictions >= threshold).float()
            actualPositives = (ratings >= threshold).float()
            actualNegatives = (ratings < threshold).float()

            TP_Val += (predictedPositives * actualPositives).sum().item()
            FP_Val += (predictedPositives * actualNegatives).sum().item()
            TN_Val += ((1 - predictedPositives) * actualNegatives).sum().item()
            FN_Val += ((1 - predictedPositives) * actualPositives).sum().item()

            if displayProgression and batch % (len(valLoader) // 10) == 0:
                valAccuracy = (TP_Val + TN_Val) / (TP_Val + TN_Val + FP_Val + FN_Val)
                valPrecision = (TP_Val / (TP_Val + FP_Val)) if (TP_Val+ FP_Val) != 0 else 0
                current = (batch + 1) * len(ratings)
                print(f"Validation Loss: {loss.item():>7f} [{current:>6d}/{(len(valLoader.dataset)):>6d}] Accuracy: {(100*valAccuracy):>3.1f} Precision: {(100*valPrecision):>3.1f}")

    valAccuracy = (TP_Val + TN_Val) / (TP_Val + TN_Val + FP_Val + FN_Val)
    valPrecision = (TP_Val / (TP_Val + FP_Val)) if (TP_Val+ FP_Val) != 0 else 0
    valLoss /= len(valLoader)
    return valLoss, valAccuracy, valPrecision

In [11]:
# Initialize the model
model = DeepNN(maxUsersId + 1, maxRecipesId + 1)
model = model.to(device)
criterion = nn.HuberLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=5, verbose=True)

batchSize = 32
trainLoader, valLoader, testLoader = setupLoaders(batchSize, trainInteractionDataset, valInteractionDataset, testInteractionDataset)

# Training
epochs = 10
trainLosses = []
valLosses = []
bestValAccurracy = 0.0
displayProgression = True

startTime = time.time()
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------------------------------------------------")
    trainLoss, trainAccuracy, trainPrecision = trainEpoch(model, trainLoader, device, criterion, optimizer, displayProgression)
    valLoss, valAccuracy, valPrecision = testEpoch(model, valLoader, device, criterion, displayProgression)

    if displayProgression:
        print()
    print(f"Training Loss: {trainLoss:.4f}, Accuracy: {(100*trainAccuracy):>3.1f} Precision: {(100*trainPrecision):>3.1f} Validation Loss: {valLoss:.4f}, Accuracy: {(100*valAccuracy):>3.1f}% Precision: {(100*valPrecision):>3.1f}%\n")

    if valAccuracy < bestValAccurracy:
        print("Early stopping due to decrease in validation accuracy!")
        break
    else:
        bestValAccurracy = valAccuracy

    scheduler.step(valLoss)

endTime = time.time()

print(f"Time: {(endTime-startTime):>.2f}")


Epoch 1
-------------------------------------------------------------------------
Training Loss: 3.9234271 [    32/698901] Accuracy: 9.4 Precision: 0.0
Training Loss: 0.0776309 [ 69920/698901] Accuracy: 74.9 Precision: 92.1
Training Loss: 0.3202780 [139808/698901] Accuracy: 83.7 Precision: 92.3
Training Loss: 0.5899310 [209696/698901] Accuracy: 86.6 Precision: 92.4
Training Loss: 0.3090078 [279584/698901] Accuracy: 88.1 Precision: 92.4
Training Loss: 0.2407153 [349472/698901] Accuracy: 89.0 Precision: 92.4
Training Loss: 0.3171371 [419360/698901] Accuracy: 89.6 Precision: 92.4
Training Loss: 0.3349264 [489248/698901] Accuracy: 90.0 Precision: 92.4
Training Loss: 0.2423636 [559136/698901] Accuracy: 90.3 Precision: 92.4
Training Loss: 0.2061665 [629024/698901] Accuracy: 90.5 Precision: 92.4

Validation Loss: 0.159120 [    32/  7023] Accuracy: 93.8 Precision: 93.8
Validation Loss: 0.315105 [   736/  7023] Accuracy: 87.1 Precision: 87.1
Validation Loss: 0.340136 [  1440/  7023] Accuracy: 8

In [12]:
testLoss, testAccuracy, testPrecision = testEpoch(model, testLoader, device, criterion, displayProgression)
print(f"Test Loss: {(testLoss):>0.4f}")
print(f"Test Accuracy: {(100*testAccuracy):>0.1f}%")
print(f"Test Precision: {(100*testPrecision):>0.1f}%")


Validation Loss: 0.148183 [    32/  7023] Accuracy: 93.8 Precision: 93.8
Validation Loss: 0.317613 [   736/  7023] Accuracy: 86.7 Precision: 87.0
Validation Loss: 0.331680 [  1440/  7023] Accuracy: 85.6 Precision: 85.8
Validation Loss: 0.282718 [  2144/  7023] Accuracy: 85.4 Precision: 85.7
Validation Loss: 0.598655 [  2848/  7023] Accuracy: 85.2 Precision: 85.5
Validation Loss: 0.329529 [  3552/  7023] Accuracy: 85.3 Precision: 85.6
Validation Loss: 0.670699 [  4256/  7023] Accuracy: 85.2 Precision: 85.3
Validation Loss: 0.562646 [  4960/  7023] Accuracy: 85.0 Precision: 85.2
Validation Loss: 0.415347 [  5664/  7023] Accuracy: 84.7 Precision: 84.8
Validation Loss: 0.505772 [  6368/  7023] Accuracy: 84.2 Precision: 84.4
Test Loss: 0.4765
Test Accuracy: 83.9%
Test Precision: 84.0%


In [14]:
saveModel = False
if saveModel:
    torch.save(model, 'model.pt')